In [11]:
from cashe import get_cashed_input
from src.data_local import MergedDataPreprocessing
from src.lstm_encoder import LSTMEmbedding
import time
from src.model_train import encode_label
import numpy as np

In [74]:
df_original = get_cashed_input()
df_original = df_original[['VISIT_ID','SERVICE_DESCRIPTION', 'SERVICE_TYPE', 'OASIS_IOS_DESCRIPTION','PROVIDER_DEPARTMENT','OUTCOME']]

In [61]:
preprocessing = MergedDataPreprocessing(df_original)
df_train, df_test = preprocessing.train_test_split()

In [64]:
train_columns = list(df_train.columns)[1:]

In [65]:
X_train = df_train[train_columns[:-1]];  y_train = df_train[train_columns[-1]].values
X_test = df_test[train_columns[:-1]];  y_test = df_test[train_columns[-1]].values

y_train = encode_label(y_train); y_test = encode_label(y_test)

In [68]:
X_train

,SERVICE_DESCRIPTION,SERVICE_TYPE,OASIS_IOS_DESCRIPTION,PROVIDER_DEPARTMENT
0,Unlisted Code,services,I.V. INFUSION OBS.,OBSTERTRIC/GYNEACOLOGIC SERVIC - نساء و ولادة
1,TSH,laboratory,TSH (THYROID STIMULATING HORMONE),E.N.T. SERVICES - انف واذن وحنجرة
3,ELICA-M CREAM 30 GM,medication-codes,ELICA-M CREAM 30 GM,OBSTERTRIC/GYNEACOLOGIC SERVIC - نساء و ولادة
4,Peripheral intravenous cannula,medical-devices,INSERTION OF AN I.V. CANNULA,EMERGENCY ROOM SERVICES - طوارى
5,Computerised tomography of brain,imaging,C.T. BRAIN WITHOUT CONTRAST,EMERGENCY ROOM SERVICES - طوارى
...,...,...,...,...
488822,BUN,laboratory,UREA NITROGEN,CARDIOLOGY SERVICES - القلب
488829,NEBILET 5MG TABLETS,medication-codes,NEBILET 5MG TABLETS,CARDIOLOGY SERVICES - القلب
488830,Unlisted Procedure Code,procedures,PLASTER STRIPS,CARDIOLOGY SERVICES - القلب
488831,SALURIN 20MG-2ML AMPOULES,medication-codes,SALURIN 20MG-2ML AMPOULES,CARDIOLOGY SERVICES - القلب


In [71]:
lstm_embedding = LSTMEmbedding()
X_train_processed = lstm_embedding.preprocess_data(X_train)
X_train_processed

array([[ 73,  70,   1, ...,   0,   0,   0],
       [157,   4, 157, ...,   0,   0,   0],
       [607, 236, 176, ...,   0,   0,   0],
       ...,
       [ 73, 161,  70, ...,   0,   0,   0],
       [565, 151, 158, ...,   0,   0,   0],
       [117,  16, 118, ...,   0,   0,   0]])

In [72]:
print('Training Model')
lstm_embedding.train_model(X_train_processed, y_train,epochs=20,batch_size=32)

print('Evaluating Model')
accuracy = lstm_embedding.evaluate_model(X_test, y_test)

print('Model Accuracy on test data: ',accuracy)

Training Model
Epoch 1/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 37s 7ms/step - accuracy: 0.5355 - loss: 0.6859
Epoch 2/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 34s 7ms/step - accuracy: 0.5617 - loss: 0.6741
Epoch 3/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - accuracy: 0.5766 - loss: 0.6690
Epoch 4/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 33s 7ms/step - accuracy: 0.5860 - loss: 0.6657
Epoch 5/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 32s 7ms/step - accuracy: 0.5959 - loss: 0.6603
Epoch 6/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 33s 7ms/step - accuracy: 0.5999 - loss: 0.6574
Epoch 7/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 33s 7ms/step - accuracy: 0.6026 - loss: 0.6551
Epoch 8/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 33s 7ms/step - accuracy: 0.6084 - loss: 0.6505
Epoch 9/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 32s 7ms/step - accuracy: 0.6091 - loss: 0.6498
Epoch 10/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 33s 7ms/step - accuracy: 0.6118 - loss: 0.6474
Epoch 11/20
4890/4890 ━━━━━━━━━━━━━━━━━━━━ 35s 7ms/step - accuracy: 0.6114 - loss: 0.6465
Epoc

In [77]:
lstm_embedding.input_shape

(37, 1)

In [78]:
X_test_vectorized = lstm_embedding.embedding_vector(X_test[:3],reload_model=False)
X_test_vectorized

array([[  32,  237,    4,    4,  342,  131,  141,    5,    6,    7,    0,
           0,    0,    0,    0,    0],
       [  43,    2,    3,  811,   78,    8,  494,  211, 1239,   43,   46,
          47,   48,   49,   50,   51],
       [ 368,  151,   23,   22,   33,    2,    3,  368,  151,   23,   22,
          33,    5,    6,    7,    0]])

In [79]:
start_time = time.time()
lstm_encoder = LSTMEmbedding()
lstm_encoder.load_model()
X_test_vectorized = lstm_embedding.embedding_vector(X_test[:10],reload_model=True)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time: {:.2f} seconds".format(execution_time))

Execution time: 0.41 seconds


In [80]:
X_test_vectorized

array([[ 55, 118, 128, 107,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [711,  35,   6,  37, 711,  35,   6,  37,   0,   0,   0,   0,   0,
          0,   0,   0],
       [346, 818, 239,  24, 132, 346, 818, 239,  24, 132,   0,   0,   0,
          0,   0,   0],
       [141, 141,  55,  57,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 34,  81,  61, 107, 570, 571, 104, 572, 107,   0,   0,   0,   0,
          0,   0,   0],
       [572, 266, 570, 571, 104, 572, 266,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [470,   5, 127, 279,   5, 146, 280,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [165, 164,   8, 165, 164,  55,  57,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 83,  78,  64,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 86,  69,  86,  69,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]])

In [82]:
def vectorize(COUNT_VALS):
      preds = lstm_encoder.embedding_vector(X_test[:COUNT_VALS],reload_model=True)
      return

def calculate_excecution():
  exs = []
  COUNT_VALS = 50
  for i in range(10):
    start_time = time.time()

    vectorize(COUNT_VALS)

    end_time = time.time()

    execution_time = end_time - start_time
    exs.append(execution_time)

  average = np.array(exs).mean()
  return average/COUNT_VALS

In [83]:
calculate_excecution()

0.002570923328399658